In [2]:
from sqlalchemy import create_engine

In [3]:
engine = create_engine('sqlite:///ecommerce.db', echo=True)

In [5]:
from sqlalchemy import text

with engine.connect() as connection:
    result = connection.execute(text("SELECT 1"))
    # Fetch and print the result
    print(result.scalar()) # This should print 1 if the connection is successful

2025-08-05 11:55:18,106 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-05 11:55:18,108 INFO sqlalchemy.engine.Engine SELECT 1
2025-08-05 11:55:18,109 INFO sqlalchemy.engine.Engine [generated in 0.00316s] ()
1
2025-08-05 11:55:18,111 INFO sqlalchemy.engine.Engine ROLLBACK
